In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
!pip install -q -U "tensorflow-text==2.8.*"
import tensorflow_text
import tensorflow as tf

     |████████████████████████████████| 4.9 MB 8.7 MB/s 
     |████████████████████████████████| 462 kB 69.5 MB/s 


In [3]:
model_name = '7_epoch_complete_bert'
drive_path = './drive/MyDrive/' + model_name

In [4]:
model_name1 = '7_epoch_bert'
drive_path1 = './drive/MyDrive/' + model_name1
model_old = tf.saved_model.load(drive_path1)


In [5]:
tf.saved_model.LoadOptions(
    allow_partial_checkpoint=False, experimental_io_device='/job:localhost',
    experimental_skip_checkpoint=False
)

In [6]:
model = tf.saved_model.load(drive_path)

In [7]:
import pandas as pd

def read_file_pos_neg(file):
  df = pd.read_csv(file)
  df_list = df.values.tolist()

  pos = []
  neg = []

  for i in df_list:
    if i[0] == 'pos':
      pos.append(i[1])
    if i[0] == 'neg':
      neg.append(i[1])
    
  return pos, neg

In [8]:
pos_valid, neg_valid = read_file_pos_neg("./drive/MyDrive/pos_valid.csv")

In [9]:
pos_valid= []
neg_valid= []

In [10]:
df = pd.read_csv("./drive/MyDrive/pos_valid.csv")
pos_valid = df.values.tolist()
len(pos_valid)

1114

In [31]:
validation_set = []
for i in pos_valid:
  validation_set.append(i)

for i in neg_valid:
  validation_set.append(i)


In [32]:
len(validation_set)

1203

In [33]:
validation_sentiment = []
for i in pos_valid:
  validation_sentiment.append(["P", i])

for i in neg_valid:
  validation_sentiment.append(["N", i])

In [34]:
def get_results(list_inps):
  scores = tf.sigmoid(model(tf.constant(list_inps)))
  results = (scores.numpy()[0][0])
  return results

In [35]:
def get_results_old(list_inps):
  scores = tf.sigmoid(model_old(tf.constant(list_inps)))
  results = (scores.numpy()[0][0])
  return results

In [70]:
get_results(["Give better instructions to make the stint worker understand the intensity and also poor hygiene in cleaning, theyve given a hard worker 1 star for cleaning"])

0.019808564

In [71]:
get_results_old(["Give better instructions to make the stint worker understand the intensity and also poor hygiene in cleaning, theyve given a hard worker 1 star for cleaning"])

0.9650419

In [36]:
false_list = []

corr = 0
incorr = 0
t = 0
for i in validation_set:  
  if get_results(i) < 0.5 :
    false_list.append(validation_sentiment[t])
  t += 1  

  
# acc = (corr/(corr+incorr)) * 100

In [48]:
false_list_new = []

corr = 0
incorr = 0
t = 0
for i in validation_set:  
  if get_results_old(i) >= 0.5 and validation_sentiment[t][0] == "P":
    corr += 1
  elif get_results_old(i) < 0.5 and validation_sentiment[t][0] == "N":
    corr += 1
  else:
    incorr += 1
    false_list_new.append(validation_sentiment[t])
  t += 1  


acc = (corr/(corr+incorr)) * 100

In [49]:
false_list = []

corr = 0
incorr = 0
t = 0
for i in validation_set:  
  if get_results(i) >= 0.5 and validation_sentiment[t][0] == "P":
    corr += 1
  elif get_results(i) < 0.5 and validation_sentiment[t][0] == "N":
    corr += 1
  else:
    incorr += 1
    false_list.append(validation_sentiment[t])
  t += 1  


accu = (corr/(corr+incorr)) * 100

In [50]:
acc

95.76059850374065

In [51]:
accu

98.33748960931005

In [64]:
(false_list_new)

[['P',
  ['Great people, would’ve been 5* but the staff toilets are a fire hazard; as once locked they cannot be unlocked from the outside.']],
 ['P', ['Staff was amazing and friendly but trays \n are heavy']],
 ['P', ['Lovely staff, but job role wasnt clear']],
 ['P',
  ['Wrong address given on Stint, should be updated to what’s on Google.']],
 ['P',
  ['Nice staff, however they need to provide moisturiser/proper washing up gloves for the KP role! I am now suffering from contact dermatitis as a result of this stint.']],
 ['P', ['It would have been great if they provided ppe like gloves and efc']],
 ['P',
  ['Good atmosphere but put me in charge of drinks which wasnt part of the description or something i was prepared for']],
 ['P',
  ['There was alot of chicken bones around, which can be uncomfortable for vegans']],
 ['P', ['Staff have to pay for the food!Lol']],
 ['P', ['Too hectic and busy']],
 ['P',
  ['Friendly staff just not much to do, was standing around for most it']],
 ['P', 

7_epoch_albert
*   S2B(N) : 94.62 acc
*   S2B : 95.75 acc

7_epoch_bert
*   S2B(N) : 94.41 acc
*   S2B : 95.95 acc




In [53]:
(false_list)

[['P',
  ['Reached 15 minutes early, had to wait 20 mins because no escort came in time, mixed instructions from employees, not allowed a 5 minut break in a 5 hour shift, lots can be improved. My guide was very helpful (mendy)']],
 ['P', ['Everything is good but broken glass  cut my hand']],
 ['P',
  ['Wrong address given on Stint, should be updated to what’s on Google.']],
 ['P',
  ['There was alot of chicken bones around, which can be uncomfortable for vegans']],
 ['P', ['It was good but 6 hours work no break and no food.']],
 ['P',
  ['This task was fairly easy to manage, consdering id had know expeience hosting before. However, the establishment exepted more orders than they could handle, leading to long wait times and disgruntled customers.']],
 ['P', ['Still just the food waste i cant hack']],
 ['P', ['did washing up for three hours which did my back in a bit']],
 ['P', ['all members are too friendly and helpful.']],
 ['P',
  ['Lovely staff. Horrible food waste. The hotel could p

Feedback with ambiguous sentiment:

['P', 'Friendly guys, enough work to get on with but not too much to handle']

 ['P', 'Friendly snd nice staff']

 ['P', 'Good service and polite staff\nWould be happy to work again']

 ['P', 'Just for ziggys singing']

 ['P', 'Listed wrong uniform']

 ['P', 'Nice friendly staff']

 ['P',
  "Proper washing up gloves would be better as normal gloves didn't stop my hands from getting wrinkles when washing up all day"]

 ['P',
  'Really good atmosphere. But working a 7 hour shift with the manager requesting me to work an additional 2 hours, i wouldve been happy to be given a short break.']

 ['P', 'Staff were lovelt free food']

 ['P',
  'Staff were very friendly. Was clear from the start as to what I had to do']

 ['P', 'Still vibes']

 ['P', 'Wrong address given on Stint, should be updated to what’s on Google.']

 ['P', 'get shift, friendly staff']

 ['P', 'good but i wasnt really needed']

 ['P',
  'the manager (i forgor her name im sorry) was an amazing help and so was the the asian lady (i forgot her name as well so so sorry). everyone was friendly though !!!']

 ['N',
  "1 star because i didn't sign up to be a runner, i sign up for washing dishes and changing bins. 2 stars for the extremely friendly kitchen staff."]

 ['N', 'Better customer service is needed']

 ['N', 'Demand is too much for low payment']

 ['N', 'Did not communicate with us at all']

 ['N',
  'Didnt show me how to run the food and had a problem with it when i got it wrong first time, other than one staff member everyone was helpful and friendly']

 ['N',
  'Entrance to the building itself takes 20 mins because of ‘security checks’. Besides food promised not given']

 ['N',
  'Honestly the place itself was beautiful however, i feel as if a lot of things should have been mentioned before hand for example, the fact you must walk up many stairs with heavy trays that barely fit through the kitchen door was very hard and not only was it exhausting it was quite physically straining and if it was mentioned i most likely would never have taken the job. If im being brutally honest i was extremely disappointed to have to take the trays up and down the stairs with little breaks i was literally sweating and out of breath and felt as if i was going to pass out. Not only this i felt a lot of pressure to which i have never felt usually from doing any stints and it was so bad my nose actually started to bleed and a few staff members walked past me and didnt even ask if i was okay. I can say that overall this was a really off putting experience i totally understand its a fast paced industry but that was too much for me to handle and if it was too hard for me to handle imagine other students feeling the same. I feel as if the stairs and heavy lifting should be noted on stint for future students.']

 ['N',
  'I washed dishes for hours straight with no gloves and no proper breaks, i believe the place was under staffed and duties were not spread equally']

 ['N',
  'In this stint, the workload was too much;  three hours of non-stop washing dishes. No food was offered. Only water was given to me. People were ok.']

 ['N',
  'It wasnt bad, staff was nice but find it rude that stints don’t get provided with a meal, especially since this is fastfood']

 ['N',
  'Need to work on hygiene and explaining things thoroughly before the shift starts instead of being annoyed with every question']

 ['N', 'No meal provided at shift and was very busy place . Staff was nice .']

 ['N', 'Rude manager']

 ['N',
  'Staff are not friendly and mature in dealing with issues regarding stints.']

 ['N',
  'This place is lovely if maybe you do a bar back shift but the Commis is hard. The kitchen is in thr basement and it HOT, this shift should come with a fitness warning because you are constantly going up and down stairs (2 flights) whilst carrying heavy items. It is not easy at all']

 ['N', 'Too much work for a one person']

 ['N', 'Unclear communication']
 
 ['N', 'Unclear instructions and unfair rating']


 Possible negatives 
 
 ['P', '10/10 and got pizza after so like 11/10'],
 ['P', 'Atmosphere is too great for working; people there are too nice'],
 ['P',
  'Christian is a great manager running a tight efficient ship. Everyone was helpful and supportive when I was getting my feet offtheh'],
 ['P', 'Even got food offered after the stint'],
 ['P',
  'Everyone is nice, but the work is quite demanding as the kitchen porter needs to run between 2 floors with trays of dishes for wash. It may not suitable for people that cannot carry heavy things.'],
 ['P', 'Everyone was beyond polite'],
 ['P', 'Excellent working vibe but the food given made me feel sick.'],
 ['P',
  'Friendly but I was assigned the kitchen dishwasher instead of the job described (not an issue though)'],
 ['P',
  'Friendly staff and good atmosphere, enough to do that i wasnt bored but not rushed off my feet either'],
 ['P',
  'Friendly staff who are willing to help, nice atmosphere, good shift length. Would go back to work again.'],
 ['P',
  'Friendly staff who made me feel confortable. I was given some complimentary food as well.'],
 ['P',
  'Friendly staff. Details of stint were true to task. However alot of the stint was spent running'],
 ['P',
  'Good stint, but needs a safer area to store bags as when i finished my bag had fallen down the back of the lockers'],
 ['P',
  'Good stint, very friendly staff and a generally helpful environment! A bit more of an introduction to what to do might have been useful as I had to ask a lot of questions. Nobody who worked there, at the front desk or in F&B knew what Stint was or the process which meant I had to explain why I was there, which was not ideal.'],
 ['P', 'Good vibes only'],
 ['P',
  'Great but if given some proper washing up gloves to use it would’ve been better'],
 ['P', 'Great staff, not much work'],
 ['P', 'Guys and gals were chill, bit hard to find on google.'],
 ['P', 'Had wrong location on the stint app'],
 ['P', 'I needed the cardio!'],
 ['P', 'I worked for an extra hour but it hasnt been added on yet'],
 ['P',
  'Incredible experience. Did 2 shifts and would have done more. Very profesional and staff was very heplful'],
 ['P',
  'Kitchen staff were extremely friendly would be more than happy to go back'],
 ['P', 'Lovely staff and managers :) heavy trays but its a good workout'],
 ['P', 'Lovely staff would go back'],
 ['P',
  'Lovely staff. Horrible food waste. The hotel could partner with shrub, a food bank, put it on too good to go or i would even happliy take those hash browns and dish them out. Also no recyling. We are living in climate emergency, we only have till 2030 to reduce our emissions by half to have a 50/50 chance of staying below a dangerous 2degrees celcius! Recyling really should be a given everywhere now, its embarrasing that places are so outdated and unattached from the pressing problem'],
 ['P', 'Nice atmosphere, not very clear responsabilities.'],
 ['P', 'Nice staff, offered free soft drinks.'],
 ['P',
  'Nice work environment but those stairs are brutal while carrying heavy trays!'],
 ['P', 'Other stints were largely unhelpful'],
 ['P', 'Past paced stint, staff are nice.'],
 ['P',
  'Paula and the staffs were amazing and helpful and i only did easy jobs'],
 ['P',
  'Putting 5 stars as usual, and stint, you know what i gonna say already 😂'],
 ['P',
  'Ran like clockwork but the 3 hours flew by in an instant. Would actually consider doing longer shifts here if offered.'],
 ['P',
  'Really nice girls, explained everything clearly, would be happy to come again'],
 ['P', 'Same views as my last shift here. Only positives to say'],
 ['P',
  'Staff was very helpful so u bever felt lost. Everyone was very welcoming and willing to help. Highly recommend. However as the QR code area doesnt have service, it took me 25 minutes to sign in therefore it would be really helpful if the code was placed at reception where there’s good service. Thank you'],
 ['P', 'Staff were v nice and helpful.'],
 ['P', 'Still just the food waste i cant hack'],
 ['P',
  'The staff are friendly, and the environment is good. However, the location on stint was wrong!'],
 ['P', 'They requested to have me back - i would like to work here again'],
 ['P',
  'To whom it may concern, \n\nI would like to thank you for having me at Terra Terra. The staff were all extremely welcoming, friendly and kind. However, I would like to call attention to the treatment I received from your staff member called Cynthia. I did briefly mention it after my shift to one of the managers, however I could see that it was a very busy period, so I did not want to take too much of his time to explain the situation. I felt very victimised and bullied by Cynthia during my shift, from the beginning until the end. \n\n* One instance is when Cynthia informed me that there weren’t any drinking glasses at the terrace and followed this with a patronising comment along the lines of: “We do have customers you know?” I feel like this was quite unfair because nobody had yet told me that there weren’t any drinking glasses left. I did later start to get into the habit of looking to see which glasses or utensils were low on stock, however this being my first shift it did naturally take a bit of time for me to do this. This is just one of the many snarky comments Cynthia said to me throughout the shift, which made me feel extremely disheartened. I voiced this to her and said that I didn’t think the extra comments were necessary. \n* This being my first shift, I made a few mistakes. I accidentally brought up unpolished glasses and asked Cynthia whether there was a possibility that I could polish them at the terrace. She replied with something like “No. You do know that this is a classy restaurant? We can’t polish upstairs. Why would you not polish it downstairs?” I deem this response to be unnecessarily depreciating because a simple comment like “No, please do it downstairs.” would have been enough for me to get the message. I feel like Cynthia did not try to understand or empathise with the fact that I was new and would naturally make a few mistakes. Her corrections were very critical in nature and greatly dispirited me. \n* On another occasion during the shift, a worker voiced that the drinking glasses required a restock and Cynthia loudly said around me “Yeah, I’ve told her multiple times.” This made me feel extremely humiliated. \n* Cynthia would also loudly sigh and roll her eyes if I sometimes did not initially comprehend the orders she was giving me or asked for her to repeat herself. I feel like she was very impatient with me and showed little support, especially considering that this was my first shift with Terra Terra. \n* After telling Cynthia discretely that I thought she was being a bit rude to me, she came to the kitchen to speak to me. At this point I informed her that at the end of my shift I would speak with the manager regarding her behaviour towards me, thinking that perhaps this would allow her to reconsider her behaviour towards me for the rest of the evening. She replied in a threatening tone: “You do know that we give you ratings too?” and attempted to escalate the situation by trying to make me go to the manager there and then. She said something like “Let’s go to the manager then.” I did not want to go, firstly because I felt extremely intimidated, and most importantly because this would pause me from doing my work which is what I was hired for. After I said once “I do not wish to discuss this here.” as I considered it extremely unprofessional and classless to argue in front of the rest of the staff, she continued to try to pressure me to go to the manager with her. This made me feel threatened and very embarrassed because many members of the kitchen staff, who I did not personally know, were nearby.\n* About half an hour later she said to me “When you want to speak to the manager let me know”. I replied saying that I would prefer to have a private conversation with the manager to which she replied something like “Why would you want to do that? You can say what you want to say around me.”  which I felt was disrespecting my decision. \n\nTo conclude, I feel like I couldn’t work and perform to the best of my ability by virtue of the constant discouragement, belittlement and discourteousness I was subject to by Cynthia. I felt extremely bullied and her behaviour put me under a lot more stress than what I was already under by being at a new workplace during the very busy and hectic Saturday evening. \n\nI wish to end this feedback on a positive note. It is important for me to stress that the rest of your staff were extremely positive, courteous and helpful, and their good nature has meant that this has otherwise been my favourite stint to date. I again thank you for having me at Terra Terra.\n\nKind regards, \n\nMaria-Magdalena Jurga'],
 ['P', 'Welcoming staff but not much for me to do'],
 ['P',
  'all good aside the water pressure - it was terrible and made it difficult to wash dishes and there were many many dishes to clean'],
 ['P', 'all members are too friendly and helpful.'],
 ['P', 'did washing up for three hours which did my back in a bit'],
 ['P', 'dope place to work'],
 ['P',
  'great place allowed me to wear shorts when i was warm. however they could have given me a better sponge to wash stuff'],
 ['P',
  'heard the manager calling us fucking idiots on our way our because i was texting my mum to let her know i finished work, all the other staff were fine']